In [1]:
from bs4 import BeautifulSoup
from decimal import *
import requests
import requests
import numpy as np
import pandas as pd
import unicodedata as ud
import re
import urllib2
import codecs


In [2]:
def save_string_field(dict, key, value, def_str, preamble):
    if len(value) > 0:
        if preamble == "":
            dict[key] = ud.normalize('NFKD', value).encode('ascii', 'ignore')
        else:
            dict[key] = preamble+value
    else: 
        dict[key] = def_str

def save_numeric_field(dict, key, value):
    if len(value) > 0:
        dict[key] = int(ud.normalize('NFKD', value).encode('ascii', 'ignore').replace(',', ''))
    else: 
        dict[key] = 0
def save_percent_field(dict, key, value):
    if len(value) > 0:
        dict[key] = int(ud.normalize('NFKD', value).encode('ascii', 'ignore').replace(',', ''))
    else: 
        dict[key] = 0

def save_billions_field(dict, key, value):
    nonBreakSpace = u'\xa0'
    value = value.strip(nonBreakSpace)
    if value.find('million') != -1:
        value = Decimal(ud.normalize('NFKD', value).encode('ascii', 'ignore').replace(',', '').\
            replace('$','').replace(' million',''))
        dict[key] = value / 1000
    elif len(value) > 0:
        dict[key] = Decimal(ud.normalize('NFKD', value).encode('ascii', 'ignore').replace(',', '').\
            replace('$','').replace(' billion',''))
    else: 
        dict[key] = 0

response = urllib2.urlopen('http://www.ttnews.com/top100/private/')
html = response.read()
soup = BeautifulSoup(html)
columns=['Rank2015', 'Rank2014', 'CompanyName', 'Link', \
            'Tractors2015', 'StraightTrucks2015', 'PickupCargoVan2015', 'Trailers2015', 'AnnualSales_Billion']
private_fleet = pd.DataFrame(data=np.zeros((0,len(columns))), columns=columns)
private_fleet_table = soup.find(id= "rankings_box")
for row in private_fleet_table.findAll('tr'):
    col = row.findAll('td')
    data_dict = {}
    j = 0
    for field in col:
#no links in private fleet page
#        hrefs = field.findAll('a')
#        for link in hrefs:
        save_string_field(data_dict, "Link", "", "", "")
        if j == 0:
            if field.get_text().isnumeric():
                save_numeric_field(data_dict, "Rank2015", field.get_text())
            else:
                break
        elif j == 1:
            save_numeric_field(data_dict, "Rank2014", field.get_text())
        elif j == 2:
            save_string_field(data_dict, "CompanyName", field.get_text(), "Unknown","")
        elif j == 3:
            save_numeric_field(data_dict, "Tractors2015", field.get_text())
        elif j == 4:
            save_numeric_field(data_dict, "StraightTrucks2015", field.get_text())
        elif j == 5:
            save_numeric_field(data_dict, "PickupCargoVan2015", field.get_text())
        elif j == 6:
            save_numeric_field(data_dict, "Trailers2015", field.get_text())
        else :
            save_billions_field(data_dict, "AnnualSales_Billion", field.get_text())
        j = j + 1
    single = pd.DataFrame([data_dict], columns=columns)
    private_fleet = private_fleet.append(single)
    print private_fleet
private_fleet.to_csv('private_fleet.csv')

   Rank2015  Rank2014   CompanyName Link  Tractors2015  StraightTrucks2015  \
0         1         1  PepsiCo Inc.               8397               14906   

   PickupCargoVan2015  Trailers2015 AnnualSales_Billion  
0                   0         11734                66.7  
   Rank2015  Rank2014   CompanyName Link  Tractors2015  StraightTrucks2015  \
0         1         1  PepsiCo Inc.               8397               14906   
0         2         2   Sysco Corp.               7647                1190   

   PickupCargoVan2015  Trailers2015 AnnualSales_Billion  
0                   0         11734                66.7  
0                 388          9577                46.5  
   Rank2015  Rank2014      CompanyName Link  Tractors2015  StraightTrucks2015  \
0         1         1     PepsiCo Inc.               8397               14906   
0         2         2      Sysco Corp.               7647                1190   
0         3         4  Wal-Mart Stores               6294                 1

/Users/satishjoshi/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
